In [1]:
### importation des librairies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.layers import RepeatVector
from tensorflow.keras.layers import *
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, BatchNormalization, Bidirectional, Flatten, concatenate
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau
tf.random.set_seed(42)
np.random.seed(42)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, GRU, Bidirectional, concatenate, Dropout, BatchNormalization, Embedding, MultiHeadAttention, GlobalAveragePooling1D, add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import StandardScaler
print(tf.__version__)

np.set_printoptions(precision=2)

2.16.2


In [2]:
from Data_Shaping import *
from models_check import *

In [3]:
### importation des données en pandas
x_train_df = pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/X_train_N1UvY30.csv")
y_train_df = pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/y_train_or6m3Ta.csv")
x_test_df=pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/X_test_m4HAPAP.csv")
x_train_df.head(120)


,obs_id,venue,order_id,action,side,price,bid,ask,bid_size,ask_size,trade,flux
0,0,4,0,A,A,0.30,0.0,0.01,100,1,False,100
1,0,4,1,A,B,-0.17,0.0,0.01,100,1,False,100
2,0,4,2,D,A,0.28,0.0,0.01,100,1,False,-100
3,0,4,3,A,A,0.30,0.0,0.01,100,1,False,100
4,0,4,4,D,A,0.37,0.0,0.01,100,1,False,-100
...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,4,11,A,B,-0.06,0.0,0.04,2,75,False,100
116,1,4,12,A,B,-0.01,0.0,0.04,2,75,False,100
117,1,4,13,A,A,0.06,0.0,0.04,2,75,False,4
118,1,2,14,A,B,-0.01,0.0,0.04,2,75,False,1


In [ ]:
print(split_data(x_train_df,y_train_df))

In [ ]:
numeric_array, order_ids, venues, actions, sides, trades=prepare_data_pipeline(x_min)
print(numeric_array)

In [ ]:
numeric_array, order_ids, venues, actions, sides, trades=prepare_data_pipeline(x_train_df)

In [4]:
def enhanced_gru_model(input_shape=(100, 11), order_id_dim=100, action_dim=3, 
                      side_dim=2, venue_dim=6, trade_dim=2, learning_rate=3e-3):
    
    # Entrées principales pour les features numériques
    inputs = Input(shape=input_shape)
    
    # Couches d'embedding avec régularisation L2
    order_id_input = Input(shape=(100,))
    order_id_embedding = Embedding(input_dim=order_id_dim, output_dim=16, 
                                 embeddings_regularizer=l2(1e-4))(order_id_input)
    
    action_input = Input(shape=(100,))
    action_embedding = Embedding(input_dim=action_dim, output_dim=8,
                               embeddings_regularizer=l2(1e-4))(action_input)
    
    side_input = Input(shape=(100,))
    side_embedding = Embedding(input_dim=side_dim, output_dim=4,
                             embeddings_regularizer=l2(1e-4))(side_input)
    
    venue_input = Input(shape=(100,))
    venue_embedding = Embedding(input_dim=venue_dim, output_dim=8,
                              embeddings_regularizer=l2(1e-4))(venue_input)
    
    trade_input = Input(shape=(100,))
    trade_embedding = Embedding(input_dim=trade_dim, output_dim=4,
                              embeddings_regularizer=l2(1e-4))(trade_input)

    # Concaténation des embeddings
    concatenated_embeddings = concatenate([
        order_id_embedding, action_embedding, side_embedding, 
        venue_embedding, trade_embedding
    ], axis=-1)
    
    # Fusion des données numériques et catégorielles
    x = concatenate([inputs, concatenated_embeddings], axis=-1)
    
    # Première couche de traitement avec skip connection
    residual = x
    x = Dense(256, activation='selu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = add([x, Dense(x.shape[-1])(residual)])  # Skip connection
    
    # Couches GRU bidirectionnelles avec skip connections
    residual = x
    x = Bidirectional(GRU(128, return_sequences=True,
                         kernel_regularizer=l2(1e-4),
                         recurrent_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = add([x, Dense(x.shape[-1])(residual)])  # Skip connection
    
    # Mécanisme d'attention multi-têtes
    x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x, x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Couches denses finales avec connexions résiduelles
    residual = x
    x = Dense(128, activation='selu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = add([x, Dense(128)(residual)])
    
    # Global average pooling pour réduire la dimension temporelle
    x = GlobalAveragePooling1D()(x)
    
    # Couche de sortie avec régularisation
    outputs = Dense(24, activation='softmax',
                   kernel_regularizer=l2(1e-4))(x)
    
    # Construction et compilation du modèle
    model = Model(
        inputs=[inputs, order_id_input, action_input, 
                side_input, venue_input, trade_input],
        outputs=outputs
    )
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [6]:
def enhanced_gru_model2(input_shape=(100, 11), order_id_dim=100, venue_dim=6, 
                      action_dim=3, side_dim=2, trade_dim=2, learning_rate=3e-3):
    """
    Modèle GRU avec attention et embeddings
    L'ordre des entrées correspond exactement à prepare_data_pipeline
    """
    # Entrées dans le même ordre que prepare_data_pipeline
    numeric_input = Input(shape=input_shape, name='numeric_input')  # Pour numeric_array
    order_id_input = Input(shape=(100,), name='order_id_input')    # Pour order_ids
    venue_input = Input(shape=(100,), name='venue_input')          # Pour venues
    action_input = Input(shape=(100,), name='action_input')        # Pour actions
    side_input = Input(shape=(100,), name='side_input')           # Pour sides
    trade_input = Input(shape=(100,), name='trade_input')         # Pour trades
    
    # Embeddings
    order_embed = Embedding(input_dim=order_id_dim, output_dim=16, 
                          embeddings_regularizer=l2(1e-4))(order_id_input)
    venue_embed = Embedding(input_dim=venue_dim, output_dim=8, 
                          embeddings_regularizer=l2(1e-4))(venue_input)
    action_embed = Embedding(input_dim=action_dim, output_dim=4, 
                           embeddings_regularizer=l2(1e-4))(action_input)
    side_embed = Embedding(input_dim=side_dim, output_dim=4, 
                          embeddings_regularizer=l2(1e-4))(side_input)
    trade_embed = Embedding(input_dim=trade_dim, output_dim=4, 
                          embeddings_regularizer=l2(1e-4))(trade_input)
    
    # Concaténation
    concat_embeddings = concatenate([
        order_embed, venue_embed, action_embed, side_embed, trade_embed
    ])
    x = concatenate([numeric_input, concat_embeddings], axis=-1)
    
    # Dense layers avec skip connections
    residual = x
    x = Dense(256, activation='selu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = add([x, Dense(x.shape[-1])(residual)])
    
    # GRU layers
    residual = x
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = add([x, Dense(x.shape[-1])(residual)])
    
    # Attention
    x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x, x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Global pooling et sortie
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation='selu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(24, activation='softmax')(x)
    
    # Construction du modèle avec les entrées dans le même ordre que prepare_data_pipeline
    model = Model(
        inputs=[numeric_input, order_id_input, venue_input, 
                action_input, side_input, trade_input],
        outputs=outputs
    )
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [7]:

model28,history28= train_model(
    model_function=lambda: enhanced_gru_model2(
        input_shape=(100, 11),
        order_id_dim=100,  
        venue_dim=6,      
        action_dim=3,     
        side_dim=2,       
        trade_dim=2,      
        learning_rate=3e-3
    ),
    X=x_train_df,
    Y=y_train_df,
    epochs=20,
    batch_size=32
)




Colonnes disponibles avant encodage: ['obs_id', 'venue', 'order_id', 'action', 'side', 'price', 'bid', 'ask', 'bid_size', 'ask_size', 'trade', 'flux', 'spread', 'order_imbalance', 'cumul_volume', 'price_change', 'price_anomaly']
Colonnes disponibles après encodage: ['obs_id', 'venue', 'order_id', 'price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux', 'spread', 'order_imbalance', 'cumul_volume', 'price_change', 'price_anomaly', 'action_encoded', 'side_encoded', 'trade_encoded']
successfull normalizayion
Colonnes disponibles avant encodage: ['obs_id', 'venue', 'order_id', 'action', 'side', 'price', 'bid', 'ask', 'bid_size', 'ask_size', 'trade', 'flux', 'spread', 'order_imbalance', 'cumul_volume', 'price_change', 'price_anomaly']
Colonnes disponibles après encodage: ['obs_id', 'venue', 'order_id', 'price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux', 'spread', 'order_imbalance', 'cumul_volume', 'price_change', 'price_anomaly', 'action_encoded', 'side_encoded', 'trade_encoded']
successful

In [1]:
def basic_gru_model(input_shape=(100, 11), order_id_dim=100, venue_dim=6, 
                   action_dim=3, side_dim=2, trade_dim=2, learning_rate=3e-3):
    """
    Modèle GRU simplifié avec embeddings de base
    """
    # Entrées numériques
    numeric_input = Input(shape=input_shape)
    
    # Embeddings simples
    order_id_input = Input(shape=(100,))
    order_id_embedding = Embedding(input_dim=order_id_dim, output_dim=8)(order_id_input)
    
    action_input = Input(shape=(100,))
    action_embedding = Embedding(input_dim=action_dim, output_dim=4)(action_input)
    
    side_input = Input(shape=(100,))
    side_embedding = Embedding(input_dim=side_dim, output_dim=2)(side_input)
    
    venue_input = Input(shape=(100,))
    venue_embedding = Embedding(input_dim=venue_dim, output_dim=3)(venue_input)
    
    trade_input = Input(shape=(100,))
    trade_embedding = Embedding(input_dim=trade_dim, output_dim=2)(trade_input)

    # Concaténation
    concatenated_embeddings = concatenate([
        order_id_embedding, action_embedding, side_embedding, 
        venue_embedding, trade_embedding
    ], axis=-1)
    
    # Fusion avec données numériques
    merged = concatenate([numeric_input, concatenated_embeddings], axis=-1)
    
    # Couches GRU simples
    x = Bidirectional(GRU(64, return_sequences=True))(merged)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Bidirectional(GRU(32))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Couches denses finales
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(24, activation='softmax')(x)
    
    # Modèle
    model = Model(
        inputs=[numeric_input, order_id_input, venue_input,
                action_input, side_input, trade_input],
        outputs=outputs
    )
    
    # Compilation
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model



In [2]:
model, history = train_model(
    model_function=lambda: basic_gru_model(),
    X=x_train_df,
    Y=y_train_df,
    epochs=50,
    batch_size=32
)

NameError: name 'train_model' is not defined

In [ ]:
x_train_final_df=process_sequences(x_train_df)
print(x_train_final_df)

In [ ]:
def prepare_data_for_rnn(df):
    action_encoded=df['action'].map({'A': 0, 'D': 1, 'U': 2})
    venue_encoded=df['venue']
    side_encoded=df['side'].map({'A': 0, 'B': 1})
    trade_encoded=df['trade'].map({True: 1, False: 0})
    return df

In [ ]:

# Définir les colonnes au niveau global
NUMERIC_COLS = ['price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux',
               'spread', 'order_imbalance', 'cumul_volume', 
               'price_change', 'price_anomaly']

CATEGORICAL_COLS = ['action', 'side', 'venue', 'trade', 'order_id']

def prepare_data_for_rnn(df):
    # 1. Encodage des variables catégorielles
    label_encoders = {}
    
    for col in CATEGORICAL_COLS:
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        label_encoders[col] = le
    
    # 3. Reshape des données en séquences de 100
    n_sequences = len(df) // 100
    
    # Préparer les inputs pour les embeddings
    categorical_inputs = {
        col: df[f'{col}_encoded'].values.reshape(n_sequences, 100)
        for col in CATEGORICAL_COLS
    }
    
    # Préparer les features numériques
    numeric_features = df[NUMERIC_COLS].values.reshape(n_sequences, 100, len(NUMERIC_COLS))
    
    return numeric_features, categorical_inputs, label_encoders

def train_rnn_model(x_train_df, y_train, input_shape):
    # Préparer les données
    numeric_features, categorical_inputs, label_encoders = prepare_data_for_rnn(x_train_df)
    
    # Obtenir les dimensions pour les embeddings
    embedding_dims = {
        'order_id': len(label_encoders['order_id'].classes_),
        'action': len(label_encoders['action'].classes_),
        'side': len(label_encoders['side'].classes_),
        'venue': len(label_encoders['venue'].classes_),
        'trade': len(label_encoders['trade'].classes_)
    }
    
    # Créer et compiler le modèle
    model = complexed_gru_reshaped2(
        input_shape=(100, len(NUMERIC_COLS)),
        order_id_dim=embedding_dims['order_id'],
        action_dim=embedding_dims['action'],
        side_dim=embedding_dims['side'],
        venue_dim=embedding_dims['venue'],
        trade_dim=embedding_dims['trade']
    )
    
    # Préparer les inputs pour l'entraînement
    model_inputs = [
        numeric_features,
        categorical_inputs['order_id'],
        categorical_inputs['action'],
        categorical_inputs['side'],
        categorical_inputs['venue'],
        categorical_inputs['trade']
    ]
    
    # S'assurer que y_train a la bonne forme
    n_sequences = len(x_train_df) // 100
    y_train = y_train[:n_sequences]  # Prendre une étiquette par séquence
    y_train_cat = to_categorical(y_train, num_classes=24)
    
    # Répéter les étiquettes pour chaque timestep si nécessaire
    y_train_cat = np.repeat(y_train_cat[:, np.newaxis, :], 100, axis=1)
    
    # Entraîner le modèle
    history = model.fit(
        model_inputs,
        y_train_cat,
        epochs=50,
        batch_size=32,
        validation_split=0.2
    )
    
    return model, history, label_encoders



In [ ]:
# Exemple d'utilisation
input_shape = (100, len(NUMERIC_COLS))  # Utilisation de la constante globale
# Assurez-vous de n'utiliser que la colonne eqt_code_cat
y_train = y_train_df['eqt_code_cat'].values  # Sélectionner uniquement la colonne de labels

model, history, encoders = train_rnn_model(x_train_final_df, y_train, input_shape)

In [ ]:
print("Valeurs uniques dans y_train:", np.unique(y_train_df))
print("\nMin:", y_train_df.min())
print("Max:", y_train_df.max())